# Descriptors

Короткое описание в [доке](https://docs.python.org/3/reference/datamodel.html#invoking-descriptors)

Должны быть методы `__get__`, `__set__`, `__del__`

Обычно когда в Python мы хотим получить доступ к атрибуту класса, то сначала мы ищем его в `__dict__` инстанса класса, потом в классе, и далее выше по цепочке наследования.

In [11]:
class BestConstant:
    def __get__(self, obj, objtype=None):
        print('Using descriptor!')
        return 73
    
class Number:
    x = 10
    y = BestConstant()  # our descriptor

In [12]:
num = Number()

In [15]:
num.x, num.y

Using descriptor!


(10, 73)

In [16]:
num.__dict__

{}

In [17]:
Number.__dict__

mappingproxy({'__module__': '__main__',
              'x': 10,
              'y': <__main__.BestConstant at 0x7fe58bc7dfa0>,
              '__dict__': <attribute '__dict__' of 'Number' objects>,
              '__weakref__': <attribute '__weakref__' of 'Number' objects>,
              '__doc__': None})

In [18]:
num.__dict__['y'] = 42

In [19]:
num.y

42

In [20]:
num.y = 32

In [21]:
num.y

32

Видим, что мы переписали значение. Дескриптор, у которого определен только get, называется non-data дескриптором

---

In [39]:
class BestConstantComplete:
    def __get__(self, obj, objtype=None):
        print('Using descriptor!')
        return obj._y
    
    def __set__(self, obj, value):
        obj._y = value
        
    def __del__(self, obj):
        del obj._y
        

class Number:
    y = BestConstantComplete()  # our new descriptor
    
    def __init__(self, x: int = 10, y: int = 42):
        self.x = x
        self.y = y

In [40]:
num = Number()

In [41]:
num.__dict__

{'x': 10, '_y': 42}

In [42]:
num.y

Using descriptor!


42

In [43]:
num.y = 73

In [44]:
num.y

Using descriptor!


73

In [45]:
num.__dict__

{'x': 10, '_y': 73}

In [46]:
num.__dict__['y'] = 100

In [47]:
num.__dict__

{'x': 10, '_y': 73, 'y': 100}

In [49]:
num.y

Using descriptor!


73

Если у дескриптора определен `__set__`, то Python при попытке достать атрибут по названию, будет доставать сначала дескриптор, даже если в `__dict__` объекта лежит что-то одноименное 

---

In [50]:
import os

In [51]:
class DirectorySize:
    
    def __get__(self, obj, objtype=None):
        return len(os.listdir(obj.dirname))
    
class Directory:
    size = DirectorySize()
    
    def __init__(self, dirname):
        self.dirname = dirname

In [54]:
! ls

data		    seminar_10_oops.ipynb	 seminar_5_regexp.ipynb
img		    seminar_1_intro.ipynb	 seminar_6_decorators.ipynb
lecture_live.ipynb  seminar_2_strings.ipynb	 seminar_7_dicts.ipynb
lecture_oop.ipynb   seminar_3_sequences.ipynb	 seminar_8_various.ipynb
README.md	    seminar_4_func_basics.ipynb  seminar_9_classes_decos.ipynb


In [59]:
a = Directory('data')

b = Directory('img')

In [60]:
a.size

3

In [64]:
b.size

1

In [66]:
! touch data/tempfile

In [68]:
a.size

4

Дескриптор вызывается при обращении к атрибуту *size*. При этом, код дескриптора выполняется каждый раз

---

**Класс Nuts**:

~можно создавать из чего угодно~

~можно индексировать по любому индексу~

~присваивать значения по индексу~

~- содержит любое поле~

~можно удалять и присваивать любые поля~

~по нeму можно итерироваться~

~имеет "красивое"  строковое представление~

~имеет формальное строковое представление~

In [173]:
class Nuts:
    
    def __init__(self, *args, **kwargs):
        self.args = args
        self.kwargs = kwargs
        
    
    def __getattr__(self, obj):
        return obj
    
    def __delattr__(self, obj):
        pass
    
    def __getitem__(self, key):
        return key
    
    def __setitem__(self, obj, value):
        pass
    
    def __delitem__(self, obj):
        pass
    
    def __iter__(self):
        return self
    
    def __next__(self):
        raise StopIteration()
    
    
    def __str__(self):
        return 'The prettiest and the nuttiest!'
    
    def __repr__(self):
        return f'Nuts({self.args}, {self.kwargs})'

In [174]:
nuts = Nuts(1, 2, 4, 10, item='gsdgdsg')

In [175]:
for item in nuts:
    print(item)

In [176]:
print(nuts)

The prettiest and the nuttiest!


In [177]:
nuts.args

(1, 2, 4, 10)

In [178]:
nuts

Nuts((1, 2, 4, 10), {'item': 'gsdgdsg'})

In [143]:
del nuts.args

In [144]:
nuts.args

'args'

In [145]:
nuts.kwargs

'kwargs'

In [146]:
nuts.afasfas

'afasfas'

In [147]:
nuts[12412412412]

12412412412

In [148]:
nuts[12] = 42

In [150]:
for item in nuts:
    print('142132')

Получилась тренировка на использование magic методов :)

Важно, чтобы методы были объявлены. Задания "разумной" логики для их работы с точки зрения синтаксиса не требуется

---

В чем могут быть проблемы при наследовании от стандартных типов

In [192]:
class DoubleDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, value * 2)
        
#     def update(self, d: dict):
#         for key, val in d.items():
#             self.__setitem__(key, val)

In [189]:
double_d = DoubleDict()

In [190]:
double_d['sdf'] = 2
double_d

{'sdf': 4}

In [191]:
double_d.update({'afaf': 24, '41': 12})
double_d

{'sdf': 4, 'afaf': 48, '41': 24}

Проигнорили setitem

In [193]:
class AnswerDict(dict):
    def __getitem__(self, key):
        return 42

In [194]:
ad = AnswerDict(a='answer')

In [196]:
ad['a']

42

In [199]:
simple_dict = {}
simple_dict.update(ad)

In [200]:
simple_dict

{'a': 'answer'}

Проигнорили getitem

Так произошло потому, что реализация метода update не использует измененные методы. Пофиксить можно наследованием от "пользовательского словаря" из модуля collections.

In [201]:
from collections import UserDict

In [202]:
class DoubleDict(UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, value * 2)

In [203]:
double_d = DoubleDict()

In [204]:
double_d['sdf'] = 2
double_d

{'sdf': 4}

In [205]:
double_d.update({'afaf': 24, '41': 12})
double_d

{'sdf': 4, 'afaf': 48, '41': 24}

In [215]:
class AnswerDict(UserDict):
    def __getitem__(self, key):
        return 42

In [213]:
ad = AnswerDict(a='answer')

In [217]:
ad['a']

42

In [221]:
ad.__dict__

{'data': {'a': 'answer'}}

In [218]:
simple_dict = {}
simple_dict.update(ad)

In [219]:
simple_dict

{'a': 42}

---

Выше мы видели, что можем объявить логику итерирования задав только один метод

In [240]:
class Digits:
    digits = '0123456789'
    
    def __getitem__(self, i):
        return self.digits[i]
    
#     def __setitem__(self, i, value):  # не будет работать, потому что строки все еще неизменяемые
#         self.digits[i] = value

In [241]:
digits = Digits()

In [242]:
for d in digits: print(d)

0
1
2
3
4
5
6
7
8
9


In [243]:
digits[1] = '0'

TypeError: 'Digits' object does not support item assignment

In [244]:
len(digits)

TypeError: object of type 'Digits' has no len()

Получается, что итерироваться можем, а длину не знаем.. Если вдруг передадим такой класс в код, который ожидает последовательность, можно получить ошибку в неожиданный момент

Можно задать явно недостающий метод

In [245]:
class Digits:
    
    def __init__(self, items:str):
        self.digits = items
    
    def __getitem__(self, i):
        return self.digits[i]
    
    def __len__(self):
        return len(self.digits)
    
#     def __setitem__(self, i, value):
#         self.digits[i] = value

In [246]:
fixed_digits = Digits('4817491724971')

In [247]:
for i in fixed_digits:
    print(i)

4
8
1
7
4
9
1
7
2
4
9
7
1


In [248]:
len(fixed_digits)

13

In [249]:
from random import choice

In [250]:
choice(fixed_digits)

'4'

---

Также можно добавить в код некоторую проверку на соответсвию интерфейсу. Например, так мы проверяем наш класс на следованию интерфейсу абстрактной последовательности

In [253]:
from collections import abc

class Digits(abc.Sequence):
    
    digits = '0123456789'
    
    def __getitem__(self, i):
        return self.digits[i]
    
    

In [252]:
digits = Digits()

TypeError: Can't instantiate abstract class Digits with abstract method __len__

И получаем ошибку при создании объекта, потому что без операции длины это не последовательность в полном смысле

In [257]:
class Digits(abc.Sequence):
    
    digits = '0123456789'
    
    def __getitem__(self, i):
        return self.digits[i]
    
    
    def __len__(self):
        return 0

In [258]:
digits = Digits()

In [259]:
len(digits)

0